In [16]:
import pandas as pd
import json
import os
import argparse as ap

ModuleNotFoundError: No module named 'utils'

In [ ]:
data = json.load(open('histogram_file.dist_analysis.json'))

df = pd.DataFrame(data["peaks"])
global_maxima = data["global_maxima"]
print(df)
print("-"*80)
print(global_maxima)
print("-"*80)
value = data["mean_freq"]
print(value)

In [ ]:
data = pd.read_csv('output-stats.tsv', sep='\t', header=0, index_col=0)
data
# data_info = data[["invalid_kmers", "non_zero_kmers", "seq_length", "%_non_zero"]]
# data_info
# data_info[["non_zero_kmers"]]

# value = data["non_zero_kmers"].sum()
# print(value)


In [ ]:
print(data.info())

In [ ]:
data = pd.read_csv('test_sequences/sequencing_summary_FAT53867_9a53b23a.txt', sep='\t', index_col=0)

#general data check
print(data.describe())
print("-"*40)


##check read_id duplicates
df1 = len(data["read_id"])
df2 = len(data["read_id"].drop_duplicates())
if df1 == df2:
    print("there really aren't any duplicated")







In [183]:
df = pd.read_csv('test_sequences/sequencing_summary_FAT53867_9a53b23a.txt', delimiter='\t')
select_data = data[["read_id", "channel", "start_time", "duration", "sequence_length_template", "mean_qscore_template",
                    "end_reason"]]
select_data.reset_index(drop=True, inplace=True)
select_data
list_of_headers = ["read_id", "channel", "start_time", "duration", "sequence_length_template", "mean_qscore_template", "end_reason"]

if not set(list_of_headers).issubset(set(df.columns)):
    raise ValueError("Error: column headers did not match expected output. check file.")
else:
    print("big pp")


big pp


In [184]:
classification = ['signal_positive',
 'data_service_unblock_mux_change',
 'signal_negative',
 'unblock_mux_change']
min_ch = 1
max_ch = 1
min_dur = 1
max_dur = max(select_data.duration)
min_q = 5
max_q = 6
min_len = 500
max_len = 5000

filtered_reads = select_data[(select_data["end_reason"].isin(classification)) & 
                             (select_data.channel.between(min_ch, max_ch)) & 
                             (select_data.duration.between(min_dur, max_dur)) &
                             (select_data.mean_qscore_template.between(min_q, max_q)) & 
                             (select_data.sequence_length_template.between(min_len, max_len))]


filtered_reads[["read_id"]]
# filtered_reads[["read_ids"]].to_csv("filtered_reads.csv", index=False)

,read_id
23091,a0c5c319-be6f-4d68-84d4-d0ef6ff7dba9
83993,ca36a63c-e7f2-4e92-8eab-3634a8e2e2b0
174987,c1982d7a-2c1c-43cd-8a39-75453c632d65
322496,b1950e6a-e609-4be0-8fed-218a38c70161


In [ ]:
def check_ont_summary_file(file_path):
    if not os.path.isfile(file_path):
        print("Error: file not found")
        return False

    with open(file_path, "r") as f:
        lines = f.readlines()

    if not lines or len(lines) < 2:
        print("Error: file is empty or has fewer than 2 lines")
        return False
    
    for line in lines[1:]:
        fields = line.strip().split("\t")
        if len(fields) != 41:
            print(f"Error: malformed line: {line.strip()}")
            return False
        try:
            int(fields[4])
        except ValueError:
            print(f"Error: invalid channel designation: {fields[4]}")
            return False
    
    return True

check_ont_summary_file("test_sequences/sequencing_summary_FAT53867_9a53b23a.txt")

In [ ]:
classes = {"stop_receiving":["signal_positive"], "unblocked":["data_service_unblock_mux_change"],
               "no_decision":["signal_negative", "unblock_mux_change"]}

classification = "no_decision"

new_classification = classes[classification]
pp = list(classes.values())
ppp = []
for x in pp:
    ppp.extend(x)
ppp

In [ ]:
with open("test_sequences/Test_br1_sal_lam_enriched.fastq", "r") as f:
    first_line = f.readline().strip()
    second_line = f.readline().strip()
    third_line = f.readline().strip()
    fourth_line = f.readline().strip()

if len(second_line) == len(fourth_line):
    print("yeahh buddy, light weight baby")

In [10]:
parser = ap.ArgumentParser(prog="sequenoscope", usage="%(prog)s <command> <argument>", description="%(prog)s version 1.0: a tool for analyzing and processing sequencing data.",
                           formatter_class= ap.RawTextHelpFormatter)

parser._positionals.title = "Commands"
# parser._positionals
parser._optionals.title = "Help"
# parser.add_argument('--foo', action='store_true', help='foo help')
subparsers = parser.add_subparsers()

analyze_parser = subparsers.add_parser('analyze', help='map reads to a target and produce a report with sequencing statistics')
plot_parser = subparsers.add_parser('plot', help='generate plots based on fastq or kmer hash files')


filter_parser = subparsers.add_parser('filter_ONT', help='filter reads from a fastq file based on a sequencing summary file')
filter_parser.usage = "sequenoscope filter_ONT --input_fastq <file.fq> --input_summary <seq_summary.txt> -o <out.fastq> [options]"
filter_parser.add_argument("--input_fastq", metavar="", required=True, nargs="+", help="[REQUIRED] Path to adaptive sequencing fastq files to process.")
filter_parser.add_argument("--input_summary", metavar="", required=True, help="[REQUIRED] Path to ONT sequencing summary file.")
filter_parser.add_argument("-o", "--output", metavar="", required=True, help="[REQUIRED] Output directory/file designation")
filter_parser.add_argument("-cls", "--classification", metavar="", choices=['unblocked', 'stop_receiving', 'no_decision'], help="a designation of the adaptive-sampling sequencing decision classification ['unblocked', 'stop_receiving', or 'no_decision']")
filter_parser.add_argument("-min_ch", "--minimum_channel", default=1, metavar="", type=int, help="a designation of the minimum channel/pore number for filtering reads")
filter_parser.add_argument("-max_ch", "--maximum_channel", default=512, metavar="", type=int, help="a designation of the maximum channel/pore number for filtering reads")
filter_parser.add_argument("-min_dur", "--minimum_duration", metavar="", type=float, help="a designation of the minimum duration of the sequencing run in SECONDS for filtering reads")
filter_parser.add_argument("-max_dur", "--maximum_duration", metavar="", type=float, help="a designation of the maximum duration of the sequencing run in SECONDS for filtering reads")
filter_parser.add_argument("-min_start", "--minimum_start_time", metavar="", type=float, help="a designation of the minimum start time of the sequencing run in SECONDS for filtering reads")
filter_parser.add_argument("-max_start", "--maximum_start_time", metavar="", type=float, help="a designation of the maximum start time of the sequencing run in SECONDS for filtering reads")
filter_parser.add_argument("-min_q", "--minimum_q_score", metavar="", type=int, help="a designation of the minimum q score for filtering reads")
filter_parser.add_argument("-max_q", "--maximum_q_score", metavar="", type=int, help="a designation of the maximum q score for filtering reads")
filter_parser.add_argument("-min_len", "--minimum_length", metavar="", type=int, help="a designation of the minimum read length for filtering reads")
filter_parser.add_argument("-max_len", "--maximum_length", metavar="", type=int, help="a designation of the maximum read length for filtering reads")
# parser.print_help()
args = parser.parse_args(["filter_ONT", "--input_fastq", "/mnt/c/Users/ameknas/Desktop/Sequenoscope/Sequenoscope/sequenoscope/analyze/test_sequences/Test_br1_sal_lam_enriched.fastq", "--input_summary", "<seq_summary.txt>", "-o", "<out.fastq>", "-cls", "no_decision"])
# filter_parser.print_help()

clas = args.input_fastq
print(clas)

['/mnt/c/Users/ameknas/Desktop/Sequenoscope/Sequenoscope/sequenoscope/analyze/test_sequences/Test_br1_sal_lam_enriched.fastq']


In [19]:
class SeqSummaryProcesser:
    parsed_report_object = None
    out_dir = None
    out_prefix = None
    classification = None
    min_ch = None
    max_ch = None
    min_dur = None
    max_dur = None
    min_start_time = None
    max_start_time = None
    min_q = None
    max_q = None
    min_len = None
    max_len = None
    status = False
    status_read_id = False
    error_messages = None
    result_files = {"filtered_read_id_list":""}
    classes = {"stop_receiving":["signal_positive"], "unblocked":["data_service_unblock_mux_change"],
               "no_decision":["signal_negative", "unblock_mux_change"], "all":["signal_positive", "data_service_unblock_mux_change", "signal_negative", "unblock_mux_change"]}

    def __init__(self, parsed_report_object, out_dir, out_prefix, classification="all", min_ch=0, max_ch=512, min_dur=0, max_dur=None,
                 min_start_time=0, max_start_time=None, min_q=0, max_q=None, min_len=0, max_len=None ):
        """
        Initalize the class with parsed_report_object, out_dir, and out_prefix

        Arguments:
            parsed_report_object: parser object
                an object that contains the parsed sequencing summary report for analysis
            out_prefix: str
                a designation of what the output files will be named
            out_dir: str
                a designation of what the output files will be stored
            classification: str
                a designation of the adaptive sampling classification based on the sequencing 
                summary file generated by ONT
            min_ch: integer
                a designation that indicates the minimum channel number in an ONT run
            max_ch: integer
                a designation that indicates the maximum channel number in an ONT run
            min_dur: integer
                a designation that indicates the minimum duration a read has undergone in an ONT run
            max_dur: integer
                a designation that indicates the maximum duration a read has undergone in an ONT run
            min_start_time: integer
                a designation that indicates the minimum start time in seconds for reads
            max_start_time: integer
                a designation that indicates the maximum start time in seconds for reads
            min_q = integer
                a designation that indicates the minimum q-score of reads in an ONT run
            max_q = integer
                a designation that indicates the maximum q-score of reads in an ONT run
            min_len = len
                a designation that indicates the minimum length of a sequence for the reads in an ONT run
            max_len = len
                a designation that indicates the maximum length of a sequence for the reads in an ONT run
        """
        self.parsed_report_object = parsed_report_object.parsed_file
        self.out_dir = out_dir
        self.out_prefix = out_prefix
        self.classification = self.classes[classification]
        self.min_ch = min_ch
        self.max_ch = max_ch
        self.min_dur = min_dur
        self.max_dur = max_dur or max(self.parsed_report_object.duration)
        self.min_start_time = min_start_time
        self.max_start_time = max_start_time or max(self.parsed_report_object.start_time)
        self.min_q = min_q
        self.max_q = max_q or max(self.parsed_report_object.mean_qscore_template)
        self.min_len = min_len
        self.max_len = max_len or max(self.parsed_report_object.sequence_length_template)
        pass

    def generate_read_ids(self):

        read_id_list = os.path.join(self.out_dir,"{}.csv".format(self.out_prefix))

        self.result_files["filtered_read_id_list"] = read_id_list

        filtered_reads = self.parsed_report_object[(self.parsed_report_object["end_reason"].isin(self.classification)) & 
                             (self.parsed_report_object.channel.between(self.min_ch, self.max_ch)) & 
                             (self.parsed_report_object.duration.between(self.min_dur, self.max_dur)) &
                             (self.parsed_report_object.start_time.between(self.min_start_time, self.max_start_time)) &
                             (self.parsed_report_object.mean_qscore_template.between(self.min_q, self.max_q)) & 
                             (self.parsed_report_object.sequence_length_template.between(self.min_len, self.max_len))]
        
        filtered_reads[["read_id"]].to_csv(read_id_list, index=False)

        self.status = self.check_files([read_id_list])
        if self.status == False:
            self.error_messages = "one or more files was not created or was empty"
            raise ValueError(str(self.error_messages))
        else:
            self.status_read_id = self.check_read_id_file(read_id_list)
            if self.status_read_id == False:
                self.error_messages = "File has less than 2 lines. No reads that match filtering criteria"
                raise ValueError(str(self.error_messages))
        pass

    def check_files(self, files_to_check):
        """
        check if the output file exists and is not empty

        Arguments:
            files_to_check: list
                list of file paths

        Returns:
            bool:
                returns True if the generated output file is found and not empty, False otherwise
        """
        if isinstance (files_to_check, str):
            files_to_check = [files_to_check]
        for f in files_to_check:
            if not os.path.isfile(f):
                return False
            elif os.path.getsize(f) == 0:
                return False
        return True
    
    def check_read_id_file(self, file):
        """
        check if the read id file has more than 1 line

        Arguments:
            file: str
                designation of read_id file path

        Returns:
            bool:
                returns True if the generated read_id file has greater than 2 lines, False otherwise
        """
        with open(file, "r") as f:
            lines = f.readlines()

        if not lines or len(lines) < 2:
            return False
        
        return True
    

class seq_summarry_process:
    result_files = {"filtered_read_id_list":"kkk"}

jjj = seq_summarry_process()

jjj.result_files["filtered_read_id_list"]

'kkk'